In [1]:
# Install Ollama inside Colab
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama server
!sudo systemctl stop ollama 2>/dev/null || true
!nohup ollama serve > /dev/null 2>&1 &

import time
time.sleep(8)

print("Ollama server started.")


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Ollama server started.


In [2]:
!ollama pull mistral


In [6]:
import sys
!pip install langchain chromadb sentence-transformers transformers torch langchain-community --quiet

if 'langchain' in sys.modules:
    print('langchain already in sys.modules, reloading...')
    import importlib
    importlib.reload(sys.modules['langchain'])

if 'langchain_community' in sys.modules:
    print('langchain_community already in sys.modules, reloading...')
    import importlib
    importlib.reload(sys.modules['langchain_community'])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires op

In [4]:
from google.colab import files
uploaded = files.upload()

print("Uploaded:", uploaded.keys())


Saving speech.txt to speech.txt
Uploaded: dict_keys(['speech.txt'])


In [7]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

SPEECH_FILE = "speech.txt"
PERSIST_DIR = "chroma_db"

# 1. Load text
loader = TextLoader(SPEECH_FILE, encoding="utf-8")
docs = loader.load()

# 2. Chunking
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=120,
)
chunks = splitter.split_documents(docs)

print("Chunks created:", len(chunks))

# 3. Embeddings (HuggingFace MiniLM)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. Create Chroma Vector Store (local)
vectordb = Chroma.from_documents(
    chunks,
    embedding=embeddings,
    persist_directory=PERSIST_DIR
)
vectordb.persist()

# 5. LLM - Ollama (Mistral 7B)
llm = Ollama(model="mistral")

retriever = vectordb.as_retriever(search_kwargs={"k": 4})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

print("RAG pipeline ready.")


Chunks created: 1


/tmp/ipython-input-136022920.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RAG pipeline ready.


/tmp/ipython-input-136022920.py:34: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
/tmp/ipython-input-136022920.py:37: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")


In [8]:
question = input("Enter your question: ")

response = qa_chain.run(question)

print("\n==== ANSWER ====\n")
print(response)


Enter your question: what is the real remedy?


/tmp/ipython-input-2449239195.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(question)



==== ANSWER ====

 The real remedy, according to the given context, is to destroy the belief in the sanctity of the shastras.
